In [4]:
!pip install -r requirements.txt

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached streamlit-1.51.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached scikeras-0.13.0-py3-none-any.whl.metadata (3.1 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)
Using cached streamlit-1.51.0-py3-none-any.whl (10.2 MB)
Using cached scikeras-0.13.0-py3-none-any.whl (26 kB)

   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]
   ---------------------------------------- 0/3 [tensorflow]



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [8]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [25]:
#Lodd the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
#Remove useless data - RowNumber, CustID, Surname
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [27]:
df.shape

(10000, 11)

In [28]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

Gender -
Female - 0
Male - 1
<br>
Geography - France - 0, Germany - 1, Spain - 2

In [29]:
#Encode Categorical Variables
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [32]:
#One hot Encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
geo_encoder = onehot_encoder.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [34]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [36]:
geo_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder.get_feature_names_out(['Geography']))

In [37]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [38]:
#Combine All the columns
df = pd.concat([df.drop(["Geography"], axis=1) , geo_df], axis = 1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [39]:
#Save the encoders and scaler as pickle files
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(le, file)

with open('onehot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

In [42]:
#Divide the dataset into independent and dependent features
X = df.drop(columns = ['Exited'], axis = 1)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [43]:
Y = df["Exited"]
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [44]:
X.shape

(10000, 12)

In [45]:
Y.shape

(10000,)

In [46]:
df.shape

(10000, 13)

In [48]:
#Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [50]:
x_train.shape, x_test.shape

((8000, 12), (2000, 12))

In [51]:
y_train.shape, y_test.shape

((8000,), (2000,))

In [52]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [53]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [54]:
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

### ANN Implementation

In [55]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [58]:
#Model Building
model = Sequential([
    Dense(64, activation = 'relu', input_shape=(x_train.shape[1], )), #HL-1
    Dense(32, activation = 'relu'), #HL-2
    Dense(1, activation ="sigmoid") #Output Layer
]
)


C:\Users\2408734\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [64]:
#Compile the model
model.compile(
    optimizer = optimizer, 
    loss = loss,
    metrics = ['accuracy']
)

In [66]:
##Set up the tensorboard
from datetime import datetime
log_dir = "logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [67]:
#Setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [68]:
history = model.fit(
    x_train, y_train, 
    validation_data = (x_test, y_test), 
    epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8384 - loss: 0.3929 - val_accuracy: 0.8530 - val_loss: 0.3522
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8556 - loss: 0.3551 - val_accuracy: 0.8575 - val_loss: 0.3455
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8545 - loss: 0.3481 - val_accuracy: 0.8555 - val_loss: 0.3459
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8600 - loss: 0.3430 - val_accuracy: 0.8575 - val_loss: 0.3453
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8591 - loss: 0.3415 - val_accuracy: 0.8535 - val_loss: 0.3540
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8612 - loss: 0.3372 - val_accuracy: 0.8555 - val_loss: 0.3464
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8604 - loss: 0.3363 - val_accuracy: 0.8565 - val_loss: 0.3474
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8633 - loss: 0.3331 - val_accu

In [69]:
model.save("model.h5")

In [70]:
## Load Tensorboard Extension
%load_ext tensorboard

In [72]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 18796), started 0:00:20 ago. (Use '!kill 18796' to kill it.)